In [5]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

import mrcnn
from mrcnn.visualize import display_instances, display_top_masks
from mrcnn.utils import extract_bboxes

from mrcnn.utils import Dataset
from matplotlib import pyplot as plt

from mrcnn.config import Config
from mrcnn.model import MaskRCNN


from mrcnn import model as modellib, utils
from PIL import Image, ImageDraw
import skimage

In [6]:
CLASS_NAMES = ["BG", "Kiste", "Packung", "Packung (oben)"]

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "inference_cfg"
	# number of classes (background + Kiste + Packung + Packung (oben))
	NUM_CLASSES = len(CLASS_NAMES)
	# Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1
	# BACKBONE = "resnet50"

In [7]:
# create config
cfg = PredictionConfig()
cfg.display() 
# define the model
model = MaskRCNN(mode='inference', model_dir='logs', config=cfg)
# load model weights
model.load_weights('logs/kaggle/mask_rcnn_custom_cfg_coco_0003.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_mask"])


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        35
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [8]:
#Show detected objects in color and all others in B&W    
def color_splash(img, mask):
    """Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]
    Returns result image.
    """
    # Make a grayscale copy of the image. The grayscale copy still
    # has 3 RGB channels, though.
    gray = skimage.color.gray2rgb(skimage.color.rgb2gray(img)) * 255
    # Copy color pixels from the original color image where mask is set
    if mask.shape[-1] > 0:
        # We're treating all instances as one, so collapse the mask into one layer
        mask = (np.sum(mask, -1, keepdims=True) >= 1)
        splash = np.where(mask, img, gray).astype(np.uint8)
    else:
        splash = gray.astype(np.uint8)
    return splash

def detect_and_color_splash(model, image_path=None):
    assert image_path
    # Run model detection and generate the color splash effect
    #print("Running on {}".format(img))
    # Read image
    img = skimage.io.imread(image_path)
    # Detect objects
    r = model.detect([img], verbose=1)[0]
    # Color splash
    splash = color_splash(img, r['masks'])
    # Save output
    file_name = "splash_{:%Y%m%dT%H%M%S}.png".format(datetime.datetime.now())
    skimage.io.imsave(file_name, splash)

    # Visualize the detected objects.
    mrcnn.visualize.display_instances(image=img, 
                                    boxes=r['rois'], 
                                    masks=r['masks'], 
                                    class_ids=r['class_ids'], 
                                    class_names=CLASS_NAMES, 
                                    scores=r['scores'])

detect_and_color_splash(model, image_path="data2/train/3228694_Box_240506_145035_color.png")

Processing 1 images
image                    shape: (480, 848, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 16)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


2024-07-11 13:00:10.730440: W tensorflow/c/c_api.cc:305] Operation '{name:'mrcnn_detection_1/Reshape_1' id:16897 op device:{requested: '', assigned: ''} def:{{{node mrcnn_detection_1/Reshape_1}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _has_manual_control_dependencies=true](mrcnn_detection_1/packed, mrcnn_detection_1/Reshape_1/shape)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-07-11 13:00:10.893516: W tensorflow/c/c_api.cc:305] Operation '{name:'mrcnn_bbox_fc_1/kernel/Assign' id:16480 op device:{requested: '', assigned: ''} def:{{{node mrcnn_bbox_fc_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](mrcnn_bbox_fc_1/kernel, mrcnn_bbox_fc_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This m

InvalidArgumentError: Graph execution error:

Detected at node 'mrcnn_class_logits_1/kernel/Initializer/stateless_random_uniform/StatelessRandomGetKeyCounter' defined at (most recent call last):
    File "/Users/basti/.pyenv/versions/miniforge3-23.11.0-0/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    File "/Users/basti/.pyenv/versions/miniforge3-23.11.0-0/lib/python3.10/runpy.py", line 86, in _run_code
    File "/Users/basti/.pyenv/versions/miniforge3-23.11.0-0/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    File "/Users/basti/.pyenv/versions/miniforge3-23.11.0-0/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    File "/Users/basti/.pyenv/versions/miniforge3-23.11.0-0/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    File "/Users/basti/.pyenv/versions/miniforge3-23.11.0-0/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    File "/Users/bas